In [1]:
import pandas as pd
import spacy
import re
import random
from recipe_scrapers import scrape_me
import csv
import pandas as pd
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import nums_from_string
from concurrent.futures import ThreadPoolExecutor, as_completed
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction

[nltk_data] Downloading package wordnet to /Users/hari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hari/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/hari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#NY times had created a recipe parser previously, so we'll be using their training dataset here

NYT_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/ingredient-phrase-tagger/master/nyt-ingredients-snapshot-2015.csv")
NYT_data = NYT_data.dropna(axis = 0, subset = ['input', 'name'])
NYT_data['input'] = NYT_data['input'].astype(str)
NYT_data['name'] = NYT_data['name'].astype(str)
NYT_data.reset_index(inplace = True)
NYT_train_data = NYT_data[0:60000].copy()
NYT_test_data = NYT_data[60001: 70000].copy()

In [3]:
wnl = WordNetLemmatizer()
vals = ['calories', 'carbohydrateContent', 'cholesterolContent', 'fatContent', 
        'fiberContent', 'proteinContent', 'saturatedFatContent', 'sodiumContent', 'sugarContent']
replace_vals = {'½':'0.5', '¼':'0.25', '¾':'0.75', '⅓':'0.33', '⅛':'0.125'}
sample_units = ['ounce', 'tablespoon', 'cup', 'pinch', 'dash', 'teaspoon', 'pound', 'clove', 'bunch',
                   'slice', 'pint', 'quart', 'gallon', 'smidgen', 'drop', 'leaf', 'stalk']

In [4]:
#Using NLTK's preprocessing packages to retain nouns and remove units from the recipe inputs

def preprocess(lines):  
    tokenized = nltk.word_tokenize(lines.replace('-', ' '))
    nouns = [wnl.lemmatize(word) for (word, pos) in nltk.pos_tag(tokenized) if (pos[0] == 'N') 
             and not (set(wnl.lemmatize(word).split())<=set(sample_units))] 
    return " ".join(nouns)

In [30]:
NYT_train_data['input'] = NYT_train_data['input'].apply(preprocess).str.lower()
NYT_test_data['input'] = NYT_test_data['input'].apply(preprocess).str.lower()

NYT_train_data.head(5)

,level_0,index,input,name,qty,range_end,unit,comment
0,0,0,butternut squash package squash,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1,1,chestnut chestnut,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,2,2,medium size onion,onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,3,3,celery coarse,celery,2.00,0.0,stalk,chopped coarse
4,4,4,oil,vegetable oil,1.50,0.0,tablespoon,NaN


In [28]:
#It needs to be modified to be feed into spacy

train_data = []
D = {}
remove_it = []
for i in range(0,len(NYT_train_data)):
    try:
        train_input = NYT_train_data['input'].iloc[i].lower()
        train_name = re.sub("[\(\[].*?[\)\]]", "", NYT_train_data['name'].iloc[i].lower())
        ing_span = list(re.search(train_name, train_input).span(0))
        ing_span.append('INGREDIENT')
        train_data.append(tuple([train_input, {'entities':[tuple(ing_span)]}]))
    except:
        remove_it.append(i)

train_data[0:5]

[('lime juice', {'entities': [(0, 10, 'INGREDIENT')]}),
 ('canola oil', {'entities': [(0, 10, 'INGREDIENT')]}),
 ('sprig cilantro', {'entities': [(6, 14, 'INGREDIENT')]}),
 ('pinch allspice', {'entities': [(6, 14, 'INGREDIENT')]}),
 ('shrimp shell', {'entities': [(0, 6, 'INGREDIENT')]})]

In [10]:
#NER training function

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [11]:
ner_model = train_spacy(train_data, 25)

/Users/hari/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(


Starting iteration 0


/Users/hari/opt/anaconda3/lib/python3.8/site-packages/thinc/neural/_classes/layernorm.py:74: RuntimeWarning: invalid value encountered in reciprocal
  d_xhat = N * dy - sum_dy - dist * var ** (-1.0) * sum_dy_dist
/Users/hari/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "fillet redfish saltwater fish" with entities "[(10, 14, 'INGREDIENT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/hari/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "herb bouquet parsley bay thyme cheesecloth" with entities "[(0, 13, 'INGREDIENT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.

{'ner': 10150.83349870116}
Starting iteration 1
{'ner': 7843.8731352400155}
Starting iteration 2
{'ner': 6992.24295245788}
Starting iteration 3
{'ner': 6834.62282536902}
Starting iteration 4
{'ner': 6763.189041783452}
Starting iteration 5
{'ner': 6733.351075279356}
Starting iteration 6
{'ner': 6728.688280357612}
Starting iteration 7
{'ner': 6675.776740410404}
Starting iteration 8
{'ner': 6784.386944055014}
Starting iteration 9
{'ner': 6676.828302753115}
Starting iteration 10
{'ner': 6936.306281837478}
Starting iteration 11
{'ner': 6855.2506828468495}
Starting iteration 12
{'ner': 6893.504319110884}
Starting iteration 13
{'ner': 6805.664210935306}
Starting iteration 14
{'ner': 6839.5646593293895}
Starting iteration 15
{'ner': 6950.107948716004}
Starting iteration 16
{'ner': 7069.2384099950605}
Starting iteration 17
{'ner': 7242.32866150697}
Starting iteration 18
{'ner': 7216.364709123974}
Starting iteration 19
{'ner': 7271.90398641402}
Starting iteration 20
{'ner': 7332.1790249097}
Star

In [12]:
#Save function to disk. This can be called later

ner_model.to_disk("ner_model")

In [13]:
test_output = []
for i in NYT_test_data.input:
    if len(ner_model(preprocess(i)).ents) > 0:
        test_output.append(ner_model(preprocess(i)).ents[0].text)
    else:
        test_output.append('')

test_output[0:20]

['salt',
 'coriander',
 'chilies',
 'salt',
 'pepper',
 'parmigiano reggiano',
 'avocado',
 'onion',
 'tomato',
 'potato',
 'tomato',
 'garlic',
 'oil',
 'pepper',
 'arugula',
 'egg',
 'olive',
 'egg',
 'thyme',
 'dijon mustard']

In [38]:
#Testing for accuracy based on sub string match. 
#If the algorithm identifies a word out of the n-grm string, its considered a match

results = {'1':0, '0':0}
for i in range(0,len(test_output)):
    if re.search(re.escape(test_output[i]),NYT_test_data.name.iloc[i].lower()):
        results['1'] += 1
    else:
        results['0'] += 1

In [43]:
accuracy = results['1']/len(test_output)*100
print("The algorithm correctly idenitfies (atleast part of) {}% of the ingredients".format(round(accuracy, 2)))

The algorithm correctly idenitfies (atleast part of) 92.78% of the ingredients


In [44]:
def serving_clean(serving_size):
    return (int(serving_size.split(' ')[0]))

In [45]:
def calorie_clean(nutrients_info):
    
    ret_vals = []
    
    for i in vals:
        ret_vals.append(float(nutrients_info[i].split(' ')[0]))
    
    return ret_vals

In [46]:
def get_name(ing):
    
    ing = re.sub(r'½|¼|¾|⅓|⅛','', ing)
    b = ner_model(preprocess(ing))
    
    return b.ents

In [47]:
def vulgar_convert(ing_frac_value):
    
    for i in replace_vals.keys():
        if i in ing_frac_value:
            ing_frac_value = ing_frac_value.replace(i, replace_vals[i])
    
    return ing_frac_value

In [48]:
def get_quant(ing):
    
    x = nums_from_string.get_nums(vulgar_convert(ing))
    
    if len(x)==1:
        quant = x[0]
    elif re.search(r'-?\(\d+\.?\d*\)*', ing):
        if len(x) == 2:
            quant = x[0]*x[1]
        else:
            quant = (x[0]+x[1])*x[2]
    elif len(x)==0:
        quant=1
    else:
        quant = x[0]+x[1]
        
    return quant

In [49]:
def get_unit(ing):
    
    words = [wnl.lemmatize(i) for i in ing.replace('(','').replace(')','').split(' ')]
    unit = 'count'
    
    for word in words:
        if word in sample_units:
            unit = word
            break
    
    return unit

In [50]:
#This is obtained from the Kaggle dataset - grabbing only the recipesIDs
#The rest of the informatin is obatined as we scrap it using the IDs from allrecipes.com

with open('Recipes.txt') as f:
    lines = f.readlines()

a = []

for i in range(1,len(lines)):
    a.append(lines[i].split(',')[0])

In [51]:
#Initializing a list of list using hearder names

recipes_df = [['RecipeID', 'Title', 'Serving_size', 'Prep_time', 'Calories', 'Carbs', 'Cholestrol', 
         'Fats', 'Fiber', 'Protein', 'SaturatedFat', 'Sodium', 'Sugar', 'Image']]
ing_df = [['RecipeID', 'IngredientName', 'Quantity', 'Unit']]
skippers = []

In [52]:
def process_items(j):
    try:
        scraper = scrape_me('https://www.allrecipes.com/recipe/' + j + '/')
        recipes_df.append([j, 
                     scraper.title(), 
                     serving_clean(scraper.yields()), 
                     int(scraper.total_time()), 
                     calorie_clean(scraper.nutrients())[0], 
                     calorie_clean(scraper.nutrients())[1],
                     calorie_clean(scraper.nutrients())[2],
                     calorie_clean(scraper.nutrients())[3],
                     calorie_clean(scraper.nutrients())[4],
                     calorie_clean(scraper.nutrients())[5],
                     calorie_clean(scraper.nutrients())[6],
                     calorie_clean(scraper.nutrients())[7],
                     calorie_clean(scraper.nutrients())[8],
                     scraper.image()])
        for i in scraper.ingredients():
            if len(get_name(i))>0:
                ing_df.append([j, get_name(i)[0].text.lower(), get_quant(i), get_unit(i)]) 
    except:
        skippers.append(j)

In [53]:
#Running processes concurrently
with ThreadPoolExecutor(max_workers = None) as executor:
    futures = [executor.submit(process_items, x) for x in a]

In [54]:
#Convert recipes list to dataframe and remove duplicates added due to multiprocessing
df_recipes = pd.DataFrame(recipes_df[1:])
df_recipes.columns = recipes_df[0]
df_recipes = df_recipes.drop_duplicates()
print(df_recipes.shape)

(40156, 14)


In [55]:
#Convert ingredients - recipe relation to dataframe and drop duplicates
df_rec2ing = pd.DataFrame(ing_df[1:])
df_rec2ing.columns = ing_df[0]
df_rec2ing = df_rec2ing.drop_duplicates()
print(df_rec2ing.shape)

(356385, 4)


In [56]:
#Create a ingredients table, add a column for ingredientID and drop duplicates
df_ingredient = df_rec2ing['IngredientName'].to_frame()
df_ingredient = df_ingredient.drop_duplicates()
df_ingredient['IngredientID'] = df_ingredient.index
df_ingredient = df_ingredient[['IngredientID', 'IngredientName']]
print(df_ingredient.shape)

(4046, 2)


In [57]:
#Replace ingredient names on recipe-ingredient relation to ingredientIDs
df_rec2ing['IngredientName'] = df_rec2ing['IngredientName'].map(df_ingredient.set_index('IngredientName')['IngredientID'])
df_rec2ing.rename(columns={"IngredientName": "IngredientID"}, inplace=True)
df_rec2ing.head()

,RecipeID,IngredientID,Quantity,Unit
0,258163,0,2.00,count
1,260453,1,0.75,cup
2,245714,2,4.00,count
3,258163,3,1.00,count
4,222388,4,3.00,pound


In [58]:
#Adding rating to the recipes table. This is obtained from the Kaggle dataset
df_full = pd.read_csv("raw-data_recipe.csv")

df_full = df_full[['recipe_id', 'aver_rate', 'review_nums']]
df_full.columns = ['RecipeID', 'Average_rating', 'No_of_reviews']
df_full = df_full.astype(str)
df_recipes = df_recipes.merge(df_full, on = 'RecipeID')

In [59]:
#Connecting to file uploaded by colleague where recipes where tagged as vegan or not

import sqlite3

dbfile = './git_repo/recipe_recommender/datasets/recipes.db'
cnx = sqlite3.connect(dbfile)

df = pd.read_sql_query("SELECT * FROM is_vegan;", cnx)
df = df.astype(str)
df.drop('index',axis=1, inplace=True)
df = df.drop_duplicates()

df.rename(columns = {'recipe_id':'RecipeID' }, inplace=True)

df_recipes = df_recipes.merge(df, on = 'RecipeID')

In [60]:
#Recipes that supposedly take not time still requires atleast a few mins to prep
df_recipes['Prep_time'][df_recipes['Prep_time'] == 0] = 10

<ipython-input-60-dafdf9068908>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recipes['Prep_time'][df_recipes['Prep_time'] == 0] = 10


In [64]:
display(df_recipes.head())
display(df_rec2ing.head())
display(df_ingredient.head())

,RecipeID,Title,Serving_size,Prep_time,Calories,Carbs,Cholestrol,Fats,Fiber,Protein,SaturatedFat,Sodium,Sugar,Image,Average_rating,No_of_reviews,isVegan
0,258163,Schnitzel Sandwich,4,40,913.5,112.3,119.0,35.3,6.0,34.1,6.7,1164.0,7.1,https://imagesvc.meredithcorp.io/v3/mm/image?u...,5.0,1,0
1,222388,Homemade Bacon,8,710,308.1,1.8,61.7,23.6,0.5,21.0,7.7,2017.1,0.1,https://imagesvc.meredithcorp.io/v3/mm/image?u...,5.0,3,0
2,245714,Potato Bacon Pizza,2,70,162.7,16.5,21.6,7.6,0.7,7.1,4.0,189.8,2.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...,4.5,2,0
3,260453,Fresh Balsamic Chicken Salad,2,26,453.7,31.0,82.5,22.6,3.0,33.6,4.3,188.1,17.9,https://imagesvc.meredithcorp.io/v3/mm/image?u...,4.0,1,0
4,103305,Hearty Chili,8,70,447.6,37.4,69.6,19.0,11.8,28.5,7.1,1445.0,10.6,https://imagesvc.meredithcorp.io/v3/mm/image?u...,4.25,3,0


,RecipeID,IngredientID,Quantity,Unit
0,258163,0,2.00,count
1,260453,1,0.75,cup
2,245714,2,4.00,count
3,258163,3,1.00,count
4,222388,4,3.00,pound


,IngredientID,IngredientName
0,0,chicken breast
1,1,vinegar
2,2,potato
3,3,salt
4,4,pork


In [68]:
file_name = './git_repo/recipe_recommender/datasets/'
df_recipes.to_csv(file_name + 'Recipes.csv', encoding='utf-8', index = None)
df_rec2ing.to_csv(file_name + 'Rec2Ing.csv', encoding='utf-8', index = None)
df_ingredient.to_csv(file_name + 'Ingredients.csv', encoding='utf-8', index = None)